In [ ]:
import pandas as pd
import json
import folium

# Load the tabular data
with open('gr_open_amzius_lytis_pilietybes-1.plain', 'r') as file:
    data = file.read()

column_names = ['Number', 'Gender', 'Age Range', 'Nationality', 'Year of declaration', 'During the quarter', 'Region code', 'Region name', 'Date']
data_frame = pd.DataFrame([x.split('|') for x in data.split('\n')], columns=column_names)

# Aggregate the data by 'Region name' and calculate the metric of interest
# For example, here we count the number of records for each region
# aggregated_data = data_frame.groupby('Region name').size().reset_index(name='count')

data_frame['Region name'] = data_frame['Region name'].replace('Rietavo sav.', 'Rietavo r. sav.')
data_frame['Region name'] = data_frame['Region name'].replace('Visagino sav.', 'Visagino m. sav.')

region_counts = data_frame['Region name'].value_counts().reset_index(name='count')

# Load the GeoJSON data
with open('lithuania_admin_level_5_simplified.json') as f:
    geojson_data = json.load(f)

# Initialize the map centered on Lithuania
m = folium.Map(location=[55.1694, 23.8813], zoom_start=7)

# Define custom thresholds
# threshold_scale = [0, 10000, 20000, 700000]  # Example thresholds

# Add the choropleth layer
folium.Choropleth(
    geo_data=geojson_data,
    name='choropleth',
    data=region_counts,
    columns=['Region name', 'count'],
    key_on='feature.properties.name',
    # fill_color='PuBu',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Population Count',
    bins=7,  # Adjust the number of bins
    # threshold_scale=threshold_scale
).add_to(m)



from folium.features import GeoJsonTooltip

# ... [your existing code to load data and create the choropleth layer] ...

# Convert your region_counts to a dictionary for easy lookup
region_dict = region_counts.set_index('Region name')['count'].to_dict()

# Iterate through GeoJSON features and add the population data
for feature in geojson_data['features']:
    region_name = feature['properties']['name']
    # Here, we directly modify the GeoJSON data structure
    feature['properties']['population'] = region_dict.get(region_name, 0)  # Use 0 or a suitable default if not found

# Initialize the map centered on Lithuania
m = folium.Map(location=[55.1694, 23.8813], zoom_start=7)

# Add the choropleth layer as before (code omitted for brevity)

# Updated tooltip to use the now available 'population' property
tooltip = GeoJsonTooltip(
    fields=['name', 'population'],
    aliases=['Region:', 'Population:'],
    localize=True,
    sticky=False,
    labels=True,
    style="""
        background-color: #F0EFEF;
        border: 2px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
    max_width=800,
)

# Add the GeoJson layer with the tooltip, no need for on_each_feature now
folium.GeoJson(
    geojson_data,
    name='Regions',
    style_function=lambda x: {'fillColor': '#ffffff', 'color': '#000000', 'fillOpacity': 0.1, 'weight': 0.1},
    tooltip=tooltip
).add_to(m)

# Save the map to an HTML file
m.save('interactive_choropleth_map.html')
